In [1]:
%pip install selenium
from selenium import webdriver


driver = webdriver.Chrome()
driver.get('https://selenium.dev/')
driver.quit()


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached selenium-4.32.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (9

In [22]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException
)

def scroll_and_screenshot_by_distance(url, scroll_pause_time=1.5, scroll_increment_js="window.scrollBy(0, window.innerHeight);"):
    """
    Navigates to a URL, handles popups, scrolls down in increments, taking a screenshot
    at each step until the bottom of the page is reached.

    Args:
        url (str): The URL of the page to scroll.
        scroll_pause_time (float): Time in seconds to wait after each scroll
                                   for content to potentially load.
        scroll_increment_js (str): JavaScript to execute for scrolling.
                                   Default scrolls by one viewport height.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # Keep browser visible to observe
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,800")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_incremental_scroll"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return

    driver = None
    try:
        driver = webdriver.Chrome(options=options)
        # driver.maximize_window() # Maximize or set specific size above

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            print("Body element loaded.")
        except TimeoutException:
            print("Page body did not become present within timeout.")
        time.sleep(3)

        # --- Handle Initial Pop-ups (Integrated robust logic) ---
        try:
            print("Looking for the first pop-up ('跳过')..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            print("Clicking '跳过'..."); driver.execute_script("arguments[0].click();", skip_button); time.sleep(0.5)
        except TimeoutException: print("First pop-up ('跳过') not found or timed out.")
        except Exception as e: print(f"Error handling first pop-up: {e}")
        try:
            print("Looking for the second pop-up ('X')..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//div[@class='xq-dialog-wrapper']//i[contains(@class,'close')]", "//i[contains(@class, 'cube-dialog-close')]", "//div[contains(@class, 'Modal_modal')]//i[contains(@class, 'Modal_closeIcon')]", "//button[@aria-label='Close']" ]
            close_button_found = False
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, xpath))); print(f"Clicking second pop-up 'X'..."); driver.execute_script("arguments[0].click();", close_button); close_button_found = True; time.sleep(0.5); break
                 except TimeoutException: continue
                 except Exception: continue
            if not close_button_found: print("Second pop-up ('X') not found or timed out.")
        except Exception as e: print(f"Error during second pop-up handling: {e}")
        # --- End Pop-up Handling ---

        # Optional: Click 'Comments' Tab
        # try:
        #     print("Looking for and clicking '评论' tab before scrolling...")
        #     tab_xpath = "//span[text()='评论' and contains(@class,'tabs__item__title')]/ancestor::div[contains(@class,'tabs__item')] | //div[contains(@class, 'tabs__item') and .//span[text()='评论']] | //div[contains(@class, 'action-bar__item') and contains(., '评论')]"
        #     comments_tab_element = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, tab_xpath)))
        #     driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", comments_tab_element)
        #     time.sleep(0.5)
        #     comments_tab_clickable = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, tab_xpath)))
        #     driver.execute_script("arguments[0].click();", comments_tab_clickable)
        #     print("Clicked '评论' tab. Waiting before scroll...")
        #     time.sleep(2.5)
        # except Exception as e_tab:
        #     print(f"Could not click '评论' tab before scrolling (maybe not needed or error): {e_tab}")

        print("\n--- Starting Incremental Scroll and Screenshot Process ---")
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_attempt = 0

        while True:
            scroll_attempt += 1
            print(f"--- Scroll Attempt #{scroll_attempt} ---")

            screenshot_path = os.path.join(screenshot_dir, f"scroll_{scroll_attempt:02d}_before.png")
            try:
                driver.save_screenshot(screenshot_path)
                print(f"Saved screenshot: {screenshot_path}")
            except Exception as e_ss:
                print(f"Failed to save screenshot {screenshot_path}: {e_ss}")

            print(f"Scrolling down using: {scroll_increment_js}")
            driver.execute_script(scroll_increment_js)

            print(f"Pausing for {scroll_pause_time} seconds...")
            time.sleep(scroll_pause_time)

            new_height = driver.execute_script("return document.body.scrollHeight")
            current_scroll_y = driver.execute_script("return window.pageYOffset || document.documentElement.scrollTop;")
            viewport_height = driver.execute_script("return window.innerHeight;")
            print(f"  Current scrollY: {round(current_scroll_y)}, Viewport height: {round(viewport_height)}, Total scrollHeight: {new_height}")

            if current_scroll_y + viewport_height >= new_height - 10:
                print("Reached the bottom of the page.")
                final_bottom_path = os.path.join(screenshot_dir, f"scroll_{scroll_attempt:02d}_at_bottom.png")
                try: driver.save_screenshot(final_bottom_path)
                except Exception as e_fin_ss: print(f"Could not save final screenshot: {e_fin_ss}")
                print(f"Saved final bottom screenshot: {final_bottom_path}")
                break
            elif scroll_attempt > 50:
                print("Reached max scroll attempts (50). Stopping.")
                break
            last_height = new_height
        print("\n--- Incremental Scroll and Screenshot Process Finished ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        # --- CORRECTED INDENTATION FOR ERROR SCREENSHOT TRY-EXCEPT ---
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error_scroll_script.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved error screenshot: {error_ss_path}")
            except Exception as e_ss_crit:
                 print(f"Could not save critical error screenshot: {e_ss_crit}")
        # --- END CORRECTION ---
    finally:
        if driver:
            print("Closing the browser...")
            driver.quit()
            print("Browser closed.")

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Incremental Scroll for URL: {target_url} ---")
    scroll_and_screenshot_by_distance(
        target_url,
        scroll_pause_time=2.0,
        scroll_increment_js="window.scrollBy(0, window.innerHeight * 0.8);"
        )
    print("--- Script Finished ---")
    print(f"Check the '{screenshot_dir}' folder for screenshots.")

--- Starting Incremental Scroll for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Created 'screenshots_incremental_scroll' directory.
Navigating to: https://xueqiu.com/5669998349/334081638
Body element loaded.
Looking for the first pop-up ('跳过')...
First pop-up ('跳过') not found or timed out.
Looking for the second pop-up ('X')...
Second pop-up ('X') not found or timed out.

--- Starting Incremental Scroll and Screenshot Process ---
--- Scroll Attempt #1 ---
Saved screenshot: screenshots_incremental_scroll\scroll_01_before.png
Scrolling down using: window.scrollBy(0, window.innerHeight * 0.8);
Pausing for 2.0 seconds...
  Current scrollY: 482, Viewport height: 602, Total scrollHeight: 4924
--- Scroll Attempt #2 ---
Saved screenshot: screenshots_incremental_scroll\scroll_02_before.png
Scrolling down using: window.scrollBy(0, window.innerHeight * 0.8);
Pausing for 2.0 seconds...
  Current scrollY: 963, Viewport height: 602, Total scrollHeight: 4924
--- Scroll At

NameError: name 'screenshot_dir' is not defined

In [26]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException
)

def scrape_post_and_comments_on_scroll(url, max_scroll_loops=15, scroll_pause_time=2.5):
    """
    Navigates to a Xueqiu post, handles popups, scrolls to load comments,
    and attempts to scrape them. Also clicks 'expand replies' and 'load more'.
    """
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_post_comments"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return {"post_content": None, "comments": []}

    driver = None
    scraped_data = {"post_content": None, "comments": []}
    unique_comment_texts_scraped = set()

    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        wait = WebDriverWait(driver, 20)
        short_wait = WebDriverWait(driver, 7)

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]")))
            print("Article body indicator loaded.")
        except TimeoutException:
            print("Article body indicator did not load. Page might be different."); driver.save_screenshot(os.path.join(screenshot_dir,"error_page_load.png")); return scraped_data
        time.sleep(2)

        # --- Handle Initial Pop-ups ---
        try:
            print("Looking for '跳过' pop-up..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            driver.execute_script("arguments[0].click();", skip_button); print("Clicked '跳过'."); time.sleep(0.5)
        except TimeoutException: print("'跳过' pop-up not found/timed out.")
        except Exception as e: print(f"Error '跳过': {e}")
        try:
            print("Looking for 'X' pop-up..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//i[contains(@class, 'cube-dialog-close')]" ]
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, xpath))); driver.execute_script("arguments[0].click();", close_button); print("Clicked 'X'."); time.sleep(0.5); break
                 except: continue
            print("Finished checking for 'X' pop-ups.") # Confirmation
        except Exception as e: print(f"Error 'X' pop-up: {e}")
        # --- End Pop-up Handling ---

        # Scrape Main Post Content
        try:
            print("Scraping main post content...")
            post_content_xpath = "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]"
            post_element = wait.until(EC.visibility_of_element_located((By.XPATH, post_content_xpath)))
            scraped_data["post_content"] = post_element.text.strip()
            print(f"Post content scraped (length: {len(scraped_data['post_content'])}).")
        except Exception as e_post:
            print(f"Error scraping post content: {e_post}")
            driver.save_screenshot(os.path.join(screenshot_dir,"error_post_scrape.png"))

        print("\n--- Starting scroll and comment extraction ---")
        last_height = driver.execute_script("return document.body.scrollHeight")
        no_new_content_strikes = 0

        for i in range(max_scroll_loops):
            print(f"--- Loop/Scroll attempt #{i+1} ---")
            initial_comment_count = len(unique_comment_texts_scraped)

            # 1. Click "查看N条回复"
            expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '条回复')]"
            try:
                expand_buttons = driver.find_elements(By.XPATH, expand_reply_xpath)
                if expand_buttons:
                    print(f"Found {len(expand_buttons)} 'Expand Replies' links.")
                    for button_idx, button in enumerate(expand_buttons):
                        try:
                            if button.is_displayed():
                                print(f"  Clicking 'Expand Replies' #{button_idx+1}..."); driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button); time.sleep(0.3)
                                driver.execute_script("arguments[0].click();", button); time.sleep(1.5)
                        except StaleElementReferenceException: print("  Stale 'Expand Replies' link, skipping.")
                        except ElementNotInteractableException: print("  'Expand Replies' link not interactable, skipping.")
                        except Exception as e_expand: print(f"  Error clicking 'Expand Replies': {e_expand}")
            except Exception as e_find_expand: print(f"Could not search for 'Expand Replies' buttons: {e_find_expand}")

            # 2. Scroll down
            print("Scrolling down...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight + 500);")
            time.sleep(scroll_pause_time)

            # 3. Attempt to scrape comments
            comment_text_xpath = "//div[@class='comment__item__main']/p"
            try:
                comment_p_tags = driver.find_elements(By.XPATH, comment_text_xpath)
                if comment_p_tags:
                    print(f"  Found {len(comment_p_tags)} potential comment <p> tags.")
                    new_comments_found_this_pass = 0
                    for p_tag in comment_p_tags:
                        try:
                            comment_text = p_tag.text.strip()
                            if comment_text and comment_text not in unique_comment_texts_scraped:
                                unique_comment_texts_scraped.add(comment_text)
                                new_comments_found_this_pass +=1
                        except StaleElementReferenceException: continue
                        except Exception as e_text: print(f"    Error getting text from a p_tag: {e_text}")
                    if new_comments_found_this_pass > 0: print(f"    Added {new_comments_found_this_pass} new unique comments.")
            except Exception as e_find: print(f"  Error finding comment <p> tags: {e_find}")

            # 4. Click "展开查看更多"
            load_more_comments_xpath = "//div[contains(@class,'more-comment') and (contains(., '展开查看更多') or contains(., '加载更多'))]"
            try:
                load_more_button = short_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_comments_xpath)))
                print("  Found '展开查看更多/加载更多' button. Clicking..."); driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button); time.sleep(0.3)
                driver.execute_script("arguments[0].click();", load_more_button); print("  Clicked '展开查看更多/加载更多'."); time.sleep(scroll_pause_time)
            except TimeoutException: pass
            except Exception as e_load_more: print(f"  Error clicking '展开查看更多': {e_load_more}")

            # 5. Check for loop termination conditions
            current_height = driver.execute_script("return document.body.scrollHeight")
            print(f"  Current total comments scraped: {len(unique_comment_texts_scraped)}. Scroll height: {current_height}")
            if len(unique_comment_texts_scraped) > initial_comment_count: no_new_content_strikes = 0
            else: no_new_content_strikes += 1

            if current_height == last_height and no_new_content_strikes >= 2 :
                print("Scroll height unchanged and no new comments for 2 strikes. Assuming all loaded or stuck.")
                break
            elif no_new_content_strikes >= 3:
                print("No new comments found for 3 consecutive strikes. Assuming all loaded.")
                break
            last_height = current_height
            if i == max_scroll_loops -1 : print("Reached max scroll loops.")
            driver.save_screenshot(os.path.join(screenshot_dir,f"loop_end_{i+1}.png"))

        scraped_data["comments"] = list(unique_comment_texts_scraped)
        print(f"\n--- Finished comment scraping. Total unique comments: {len(scraped_data['comments'])} ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        # --- CORRECTED INDENTATION FOR ERROR SCREENSHOT TRY-EXCEPT ---
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved critical error screenshot.")
            except Exception as e_ss_crit:
                 print(f"Could not save critical error screenshot: {e_ss_crit}")
        # --- END CORRECTION ---
    finally:
        if driver:
            print("Closing the browser...")
            driver.quit()
            print("Browser closed.")
    return scraped_data

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Scraper for URL: {target_url} ---")

    data = scrape_post_and_comments_on_scroll(target_url, max_scroll_loops=10, scroll_pause_time=3.0)

    print("\n" + "="*30); print("      Scraped Data Summary"); print("="*30)
    if data["post_content"]:
        print("\n--- Main Post ---")
        print(data["post_content"][:500] + "..." if len(data["post_content"]) > 500 else data["post_content"])
    else:
        print("\n>>> Main post content not scraped. <<<")

    if data["comments"]:
        print(f"\n--- Comments ({len(data['comments'])}) ---")
        for i, comment in enumerate(data["comments"][:20]): # Print first 20
            print(f"{i+1}. {comment[:150]}" + "..." if len(comment)>150 else comment)
        if len(data["comments"]) > 20:
            print(f"... and {len(data['comments']) - 20} more comments.")
    else:
        print("\n>>> No comments were scraped. <<<")
    print("\n" + "="*30)
    print(f"Check console logs and '{screenshot_dir}' folder for details.")

--- Starting Scraper for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Navigating to: https://xueqiu.com/5669998349/334081638
Article body indicator loaded.
Looking for '跳过' pop-up...
'跳过' pop-up not found/timed out.
Looking for 'X' pop-up...
Finished checking for 'X' pop-ups.
Scraping main post content...
Post content scraped (length: 627).

--- Starting scroll and comment extraction ---
--- Loop/Scroll attempt #1 ---
Scrolling down...
  Found 18 potential comment <p> tags.
    Added 2 new unique comments.
  Current total comments scraped: 2. Scroll height: 4924
--- Loop/Scroll attempt #2 ---
Scrolling down...
  Found 18 potential comment <p> tags.
  Current total comments scraped: 2. Scroll height: 4924
--- Loop/Scroll attempt #3 ---
Scrolling down...
  Found 18 potential comment <p> tags.
  Current total comments scraped: 2. Scroll height: 4924
Scroll height unchanged and no new comments for 2 strikes. Assuming all loaded or stuck.

--- Finished comment sc

NameError: name 'screenshot_dir' is not defined

In [27]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
    ElementNotInteractableException
)

def scrape_post_and_all_comments(url, max_main_loops=15, scroll_pause_time=2.5):
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_post_all_comments"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return {"post_content": None, "comments": []}

    driver = None
    scraped_data = {"post_content": None, "comments": []}
    unique_comment_texts_scraped = set()

    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        wait = WebDriverWait(driver, 20)
        short_wait = WebDriverWait(driver, 5) # Shorter wait for elements within loops

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]")))
            print("Article body indicator loaded.")
        except TimeoutException:
            print("Article body indicator did not load."); driver.save_screenshot(os.path.join(screenshot_dir,"error_page_load.png")); return scraped_data
        time.sleep(2)

        # --- Handle Initial Pop-ups ---
        try:
            print("Looking for '跳过' pop-up..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            driver.execute_script("arguments[0].click();", skip_button); print("Clicked '跳过'."); time.sleep(0.5)
        except TimeoutException: print("'跳过' pop-up not found/timed out.")
        except Exception as e: print(f"Error '跳过': {e}")
        try:
            print("Looking for 'X' pop-up..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//i[contains(@class, 'cube-dialog-close')]" ]
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, xpath))); driver.execute_script("arguments[0].click();", close_button); print("Clicked 'X'."); time.sleep(0.5); break
                 except: continue
            print("Finished checking for 'X' pop-ups.")
        except Exception as e: print(f"Error 'X' pop-up: {e}")

        # Scrape Main Post Content
        try:
            print("Scraping main post content...")
            post_content_xpath = "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]"
            post_element = wait.until(EC.visibility_of_element_located((By.XPATH, post_content_xpath)))
            scraped_data["post_content"] = post_element.text.strip()
            print(f"Post content scraped (length: {len(scraped_data['post_content'])}).")
        except Exception as e_post:
            print(f"Error scraping post content: {e_post}")
            driver.save_screenshot(os.path.join(screenshot_dir,"error_post_scrape.png"))

        print("\n--- Starting scroll and comment extraction ---")
        last_height = driver.execute_script("return document.body.scrollHeight")
        no_new_actions_or_comments_strikes = 0

        for i in range(max_main_loops):
            print(f"--- Main Loop Iteration #{i+1} ---")
            action_taken_this_loop = False
            comments_found_before_interactions = len(unique_comment_texts_scraped)

            # 1. Click ALL visible "查看N条回复" (Expand Replies)
            expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '条回复')]"
            # Loop to click these as new ones might appear after expanding others
            while True:
                clicked_an_expand_button_this_pass = False
                try:
                    expand_buttons = driver.find_elements(By.XPATH, expand_reply_xpath)
                    if not expand_buttons: # print("  No 'Expand Replies' links found this pass.");
                        break

                    # Filter only visible buttons before attempting to click
                    visible_expand_buttons = [b for b in expand_buttons if b.is_displayed()]
                    if not visible_expand_buttons: # print("  No *visible* 'Expand Replies' links.");
                        break

                    print(f"  Found {len(visible_expand_buttons)} visible 'Expand Replies' links to click.")
                    for button in visible_expand_buttons:
                        try:
                            # print("    Scrolling to 'Expand Replies' and clicking...")
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", button)
                            time.sleep(0.5) # Wait for scroll
                            # Use a short wait for clickability
                            button_to_click = short_wait.until(EC.element_to_be_clickable(button))
                            driver.execute_script("arguments[0].click();", button_to_click) # JS click
                            # button_to_click.click() # Regular click
                            print(f"    Clicked 'Expand Replies': {button.text[:20]}")
                            action_taken_this_loop = True
                            clicked_an_expand_button_this_pass = True
                            time.sleep(1.5) # Wait for replies to load
                            # After clicking, the DOM might change, so we might need to re-find buttons in the next 'while True' iteration
                        except StaleElementReferenceException: print("    Stale 'Expand Replies' link during click, will re-evaluate."); break # Break inner for to re-find
                        except ElementNotInteractableException: print("    'Expand Replies' link not interactable, might be covered or disabled.")
                        except TimeoutException: print("    Timeout waiting for 'Expand Replies' to be clickable.")
                        except Exception as e_expand: print(f"    Error clicking one 'Expand Replies': {e_expand}")
                    if not clicked_an_expand_button_this_pass: # No visible ones were successfully clicked
                        break
                except Exception as e_find_expand: print(f"  Error finding 'Expand Replies': {e_find_expand}"); break
                if not clicked_an_expand_button_this_pass: break # If loop completes without any clicks

            # 2. Scroll down
            print("  Scrolling down...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)

            # 3. Attempt to scrape comments
            comment_text_xpath = "//div[@class='comment__item__main']/p" # This XPath seems correct
            try:
                comment_p_tags = driver.find_elements(By.XPATH, comment_text_xpath)
                if comment_p_tags:
                    # print(f"  Found {len(comment_p_tags)} potential comment <p> tags on this pass.")
                    new_comments_this_pass = 0
                    for p_tag in comment_p_tags:
                        try:
                            comment_text = p_tag.text.strip()
                            if comment_text and comment_text not in unique_comment_texts_scraped:
                                unique_comment_texts_scraped.add(comment_text)
                                new_comments_this_pass += 1
                        except StaleElementReferenceException: continue
                        except Exception: continue # Ignore errors for individual comment text retrieval
                    if new_comments_this_pass > 0:
                        print(f"    Added {new_comments_this_pass} new unique comments this pass.")
                        action_taken_this_loop = True # Finding new comments is an action
            except Exception as e_find_comments: print(f"  Error finding comment <p> tags: {e_find_comments}")

            # 4. Click "展开查看更多" (Load More Main Comments)
            load_more_comments_xpath = "//div[contains(@class,'more-comment') and (contains(., '展开查看更多') or contains(., '加载更多'))]"
            try:
                # Use short_wait for this button as it might appear/disappear
                load_more_button = short_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_comments_xpath)))
                print("  Found '展开查看更多/加载更多' button. Clicking...")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", load_more_button); time.sleep(0.3)
                driver.execute_script("arguments[0].click();", load_more_button)
                print("    Clicked '展开查看更多/加载更多'.")
                action_taken_this_loop = True
                time.sleep(scroll_pause_time + 1) # Wait longer after this action
            except TimeoutException: print("  '展开查看更多/加载更多' button not found or not clickable this pass.")
            except Exception as e_load_more: print(f"  Error clicking '展开查看更多': {e_load_more}")

            # 5. Check for loop termination conditions
            current_height = driver.execute_script("return document.body.scrollHeight")
            print(f"  Loop {i+1} end. Total unique comments: {len(unique_comment_texts_scraped)}. Scroll height: {current_height}. Last height: {last_height}")

            if not action_taken_this_loop and len(unique_comment_texts_scraped) == comments_found_before_interactions:
                no_new_actions_or_comments_strikes += 1
                print(f"  No new actions or comments strike: {no_new_actions_or_comments_strikes}")
            else:
                no_new_actions_or_comments_strikes = 0 # Reset if something happened

            if no_new_actions_or_comments_strikes >= 2:
                print("No new comments found and no interaction buttons successfully clicked for 2 consecutive loops. Assuming completion.")
                break

            last_height = current_height
            if i == max_main_loops - 1: print("Reached max main loops.")
            driver.save_screenshot(os.path.join(screenshot_dir,f"main_loop_end_{i+1}.png"))

        scraped_data["comments"] = list(unique_comment_texts_scraped)
        print(f"\n--- Finished comment scraping. Total unique comments: {len(scraped_data['comments'])} ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved critical error screenshot.")
            except Exception as e_ss_crit:
                 print(f"Could not save critical error screenshot: {e_ss_crit}")
    finally:
        if driver: print("Closing the browser..."); driver.quit(); print("Browser closed.")
    return scraped_data

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Scraper for URL: {target_url} ---")

    data = scrape_post_and_all_comments(target_url, max_main_loops=10, scroll_pause_time=2.5)

    print("\n" + "="*30); print("      Scraped Data Summary"); print("="*30)
    if data["post_content"]:
        print("\n--- Main Post ---")
        print(data["post_content"]) # Print full post content
    else:
        print("\n>>> Main post content not scraped. <<<")

    if data["comments"]:
        print(f"\n--- Comments ({len(data['comments'])}) ---")
        for i, comment in enumerate(data["comments"]):
            print(f"{i+1}. {comment}") # Print full comment
    else:
        print("\n>>> No comments were scraped. <<<")
    print("\n" + "="*30)
    print(f"Check console logs and folder for details.")

--- Starting Scraper for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Created 'screenshots_post_all_comments' directory.
Navigating to: https://xueqiu.com/5669998349/334081638
Article body indicator loaded.
Looking for '跳过' pop-up...
'跳过' pop-up not found/timed out.
Looking for 'X' pop-up...
Finished checking for 'X' pop-ups.
Scraping main post content...
Post content scraped (length: 706).

--- Starting scroll and comment extraction ---
--- Main Loop Iteration #1 ---
  Scrolling down...
    Added 18 new unique comments this pass.
  '展开查看更多/加载更多' button not found or not clickable this pass.
  Loop 1 end. Total unique comments: 18. Scroll height: 4924. Last height: 4924
--- Main Loop Iteration #2 ---
  Scrolling down...
  '展开查看更多/加载更多' button not found or not clickable this pass.
  Loop 2 end. Total unique comments: 18. Scroll height: 4924. Last height: 4924
  No new actions or comments strike: 1
--- Main Loop Iteration #3 ---
  Scrolling down...
  '展开查看更多/加载

In [28]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
    ElementNotInteractableException
)

def scrape_post_and_all_comments(url, max_main_loops=20, scroll_pause_time=2.5): # Increased max_main_loops
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_post_all_comments"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return {"post_content": None, "comments": []}

    driver = None
    scraped_data = {"post_content": None, "comments": []}
    unique_comment_texts_scraped = set()

    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        wait = WebDriverWait(driver, 20)
        # Shorter wait for elements that appear/disappear within loops
        interaction_wait = WebDriverWait(driver, 7)

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]")))
            print("Article body indicator loaded.")
        except TimeoutException:
            print("Article body indicator did not load."); driver.save_screenshot(os.path.join(screenshot_dir,"error_page_load.png")); return scraped_data
        time.sleep(2)

        # --- Handle Initial Pop-ups ---
        try:
            print("Looking for '跳过' pop-up..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            driver.execute_script("arguments[0].click();", skip_button); print("Clicked '跳过'."); time.sleep(0.5)
        except TimeoutException: print("'跳过' pop-up not found/timed out.")
        except Exception as e: print(f"Error '跳过': {e}")
        try:
            print("Looking for 'X' pop-up..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//i[contains(@class, 'cube-dialog-close')]" ]
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, xpath))); driver.execute_script("arguments[0].click();", close_button); print("Clicked 'X'."); time.sleep(0.5); break
                 except: continue
            print("Finished checking for 'X' pop-ups.")
        except Exception as e: print(f"Error 'X' pop-up: {e}")

        # Scrape Main Post Content
        try:
            print("Scraping main post content...")
            post_content_xpath = "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]"
            post_element = wait.until(EC.visibility_of_element_located((By.XPATH, post_content_xpath)))
            scraped_data["post_content"] = post_element.text.strip()
            print(f"Post content scraped (length: {len(scraped_data['post_content'])}).")
        except Exception as e_post:
            print(f"Error scraping post content: {e_post}")
            driver.save_screenshot(os.path.join(screenshot_dir,"error_post_scrape.png"))

        print("\n--- Starting scroll and comment extraction ---")
        last_total_comments = -1 # Initialize to a value different from initial count

        for i in range(max_main_loops):
            print(f"--- Main Loop Iteration #{i+1} ---")
            action_taken_this_loop = False
            comments_at_loop_start = len(unique_comment_texts_scraped)

            # 1. Click ALL visible "查看N条回复" (Expand Replies)
            expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '条回复')]"
            # Inner loop to keep clicking expand replies as long as new ones appear or are clickable
            expand_attempts = 0
            while expand_attempts < 5: # Limit attempts to avoid infinite loop if something goes wrong
                expand_attempts += 1
                clicked_an_expand_button_this_pass = False
                try:
                    # Re-find elements each time as DOM changes
                    visible_expand_buttons = [b for b in driver.find_elements(By.XPATH, expand_reply_xpath) if b.is_displayed()]
                    if not visible_expand_buttons: break # No more visible expand buttons

                    print(f"  Found {len(visible_expand_buttons)} visible 'Expand Replies' links.")
                    for button in visible_expand_buttons:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", button); time.sleep(0.4)
                            # Using a short wait for the specific button to ensure it's ready
                            button_to_click = WebDriverWait(driver, 3).until(EC.element_to_be_clickable(button))
                            driver.execute_script("arguments[0].click();", button_to_click)
                            print(f"    Clicked 'Expand Replies': {button.text[:20]}")
                            action_taken_this_loop = True
                            clicked_an_expand_button_this_pass = True
                            time.sleep(1.5) # Wait for replies
                        except (StaleElementReferenceException, TimeoutException, ElementNotInteractableException): continue # Try next button or re-evaluate
                        except Exception as e_expand: print(f"    Error clicking one 'Expand Replies': {e_expand}")
                    if not clicked_an_expand_button_this_pass: break # No more were clicked this pass
                except Exception as e_find_expand: print(f"  Error finding 'Expand Replies': {e_find_expand}"); break


            # 2. Scroll down (helps reveal "Load More" and more comments)
            print("  Scrolling down...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time) # Allow content to load after scroll

            # 3. Attempt to scrape comments (after potential expansions and scroll)
            comment_text_xpath = "//div[@class='comment__item__main']/p"
            try:
                comment_p_tags = driver.find_elements(By.XPATH, comment_text_xpath)
                new_comments_this_pass = 0
                for p_tag in comment_p_tags:
                    try:
                        comment_text = p_tag.text.strip()
                        if comment_text and comment_text not in unique_comment_texts_scraped:
                            unique_comment_texts_scraped.add(comment_text)
                            new_comments_this_pass += 1
                    except StaleElementReferenceException: continue
                if new_comments_this_pass > 0:
                    print(f"    Added {new_comments_this_pass} new unique comments from page.")
                    action_taken_this_loop = True
            except Exception as e_find_comments: print(f"  Error finding comment <p> tags: {e_find_comments}")


            # 4. Click "展开查看更多" (Load More Main Comments)
            # UPDATED XPATH based on your screenshot:
            load_more_comments_xpath = "//a[@class='show_more' and .//span[text()='展开查看更多']]"
            try:
                # Use interaction_wait for this button
                load_more_button = interaction_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_comments_xpath)))
                print("  Found '展开查看更多' button. Clicking...")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", load_more_button); time.sleep(0.4)
                driver.execute_script("arguments[0].click();", load_more_button)
                print("    Clicked '展开查看更多'.")
                action_taken_this_loop = True
                time.sleep(scroll_pause_time + 0.5) # Wait longer after this significant action
            except TimeoutException:
                print("  '展开查看更多' button not found or not clickable this pass (might be all loaded).")
            except Exception as e_load_more:
                print(f"  Error clicking '展开查看更多': {e_load_more}")


            # 5. Check for loop termination conditions
            current_total_comments = len(unique_comment_texts_scraped)
            print(f"  Loop {i+1} end. Total unique comments: {current_total_comments}. Previously: {last_total_comments}")

            if not action_taken_this_loop and current_total_comments == last_total_comments:
                print("No actions taken (no buttons clicked, no new comments found) and comment count unchanged. Assuming completion.")
                break
            
            last_total_comments = current_total_comments
            if i == max_main_loops - 1: print("Reached max main loops.")
            driver.save_screenshot(os.path.join(screenshot_dir,f"main_loop_end_{i+1}.png"))

        scraped_data["comments"] = list(unique_comment_texts_scraped)
        print(f"\n--- Finished comment scraping. Total unique comments: {len(scraped_data['comments'])} ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        if driver:
            try:
                error_ss_path = os.path.join(screenshot_dir, "critical_error.png")
                driver.save_screenshot(error_ss_path)
                print(f"Saved critical error screenshot.")
            except Exception as e_ss_crit:
                 print(f"Could not save critical error screenshot: {e_ss_crit}")
    finally:
        if driver: print("Closing the browser..."); driver.quit(); print("Browser closed.")
    return scraped_data

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Scraper for URL: {target_url} ---")

    data = scrape_post_and_all_comments(target_url, max_main_loops=20, scroll_pause_time=2.5) # Increased max_main_loops

    print("\n" + "="*30); print("      Scraped Data Summary"); print("="*30)
    if data["post_content"]:
        print("\n--- Main Post ---"); print(data["post_content"])
    else: print("\n>>> Main post content not scraped. <<<")

    if data["comments"]:
        print(f"\n--- Comments ({len(data['comments'])}) ---")
        for i, comment in enumerate(data["comments"]): print(f"{i+1}. {comment}")
    else: print("\n>>> No comments were scraped. <<<")
    print("\n" + "="*30); print(f"Check console logs and folder for details.")

--- Starting Scraper for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Navigating to: https://xueqiu.com/5669998349/334081638
Article body indicator loaded.
Looking for '跳过' pop-up...
'跳过' pop-up not found/timed out.
Looking for 'X' pop-up...
Finished checking for 'X' pop-ups.
Scraping main post content...
Post content scraped (length: 706).

--- Starting scroll and comment extraction ---
--- Main Loop Iteration #1 ---
  Scrolling down...
    Added 18 new unique comments from page.
  Found '展开查看更多' button. Clicking...
    Clicked '展开查看更多'.
  Loop 1 end. Total unique comments: 18. Previously: -1
--- Main Loop Iteration #2 ---
  Scrolling down...
    Added 15 new unique comments from page.
  '展开查看更多' button not found or not clickable this pass (might be all loaded).
  Loop 2 end. Total unique comments: 33. Previously: 18
--- Main Loop Iteration #3 ---
  Scrolling down...
  '展开查看更多' button not found or not clickable this pass (might be all loaded).
  Loop 3 end.

In [29]:
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
    ElementNotInteractableException
)

def scrape_post_comments_with_authors(url, max_main_loops=20, scroll_pause_time=2.5):
    print("Setting up WebDriver...")
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu"); options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage"); options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-notifications"); options.add_argument("--lang=zh-CN")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    screenshot_dir = "screenshots_post_authors_comments"
    if not os.path.exists(screenshot_dir):
        try: os.makedirs(screenshot_dir); print(f"Created '{screenshot_dir}' directory.")
        except OSError as e: print(f"Error creating screenshot directory: {e}"); return {"post_author": None, "post_content": None, "comments": []}

    driver = None
    # Store comments as dictionaries: {"author_name": "...", "author_id": "...", "text": "..."}
    scraped_data = {"post_author_name": None, "post_author_id": None, "post_content": None, "comments": []}
    # Use a set of tuples (author_id, comment_text_start) to identify unique comments
    # This helps if a user posts the exact same comment twice (rare but possible)
    # or if our scraping picks up the same comment multiple times due to DOM re-renders.
    unique_comments_tracker = set()


    try:
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()

        wait = WebDriverWait(driver, 20)
        interaction_wait = WebDriverWait(driver, 7)

        print(f"Navigating to: {url}")
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'article__author')]"))) # Wait for author block
            print("Article page indicators loaded.")
        except TimeoutException:
            print("Article page indicators did not load."); driver.save_screenshot(os.path.join(screenshot_dir,"error_page_load.png")); return scraped_data
        time.sleep(2)

        # --- Handle Initial Pop-ups ---
        try:
            print("Looking for '跳过' pop-up..."); skip_xpath = "//span[text()='跳过'] | //button[contains(.,'跳过')]"; skip_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, skip_xpath)))
            driver.execute_script("arguments[0].click();", skip_button); print("Clicked '跳过'."); time.sleep(0.5)
        except TimeoutException: print("'跳过' pop-up not found/timed out.")
        except Exception as e: print(f"Error '跳过': {e}")
        try:
            print("Looking for 'X' pop-up..."); close_xpaths = [ "//div[contains(@class,'modal-wrapper')]//i[contains(@class,'icon-close')]", "//i[contains(@class, 'cube-dialog-close')]" ]
            for xpath in close_xpaths:
                 try: close_button = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.XPATH, xpath))); driver.execute_script("arguments[0].click();", close_button); print("Clicked 'X'."); time.sleep(0.5); break
                 except: continue
            print("Finished checking for 'X' pop-ups.")
        except Exception as e: print(f"Error 'X' pop-up: {e}")


        # Scrape Main Post Author and Content
        try:
            print("Scraping main post author and content...")
            # XPath for author link - LIKELY NEEDS INSPECTION AND ADJUSTMENT
            # Look for an <a> tag with class 'name' inside 'article__author' or similar
            post_author_link_xpath = "//div[contains(@class, 'article__author')]//a[contains(@class,'name')]"
            author_link_element = wait.until(EC.visibility_of_element_located((By.XPATH, post_author_link_xpath)))
            scraped_data["post_author_name"] = author_link_element.text.strip()
            author_href = author_link_element.get_attribute('href')
            # Extract ID from href like "https://xueqiu.com/u/USERID" or "/USERID"
            if author_href:
                parts = author_href.split('/')
                scraped_data["post_author_id"] = parts[-1] if parts[-1] else parts[-2] # Handles trailing slash
            print(f"Post Author: {scraped_data['post_author_name']} (ID: {scraped_data['post_author_id']})")

            post_content_xpath = "//div[contains(@class, 'article__bd__detail')] | //div[contains(@class, 'article__content')]"
            post_element = driver.find_element(By.XPATH, post_content_xpath) # Already waited for author, so content should be there
            scraped_data["post_content"] = post_element.text.strip()
            print(f"Post content scraped (length: {len(scraped_data['post_content'])}).")
        except Exception as e_post:
            print(f"Error scraping post author or content: {e_post}")
            driver.save_screenshot(os.path.join(screenshot_dir,"error_post_scrape.png"))


        print("\n--- Starting scroll and comment extraction ---")
        last_total_comments_in_set = -1

        for i in range(max_main_loops):
            print(f"--- Main Loop Iteration #{i+1} ---")
            action_taken_this_loop = False
            comments_found_before_interactions = len(unique_comments_tracker)

            # 1. Click ALL visible "查看N条回复"
            expand_reply_xpath = "//a[contains(text(), '查看') and contains(text(), '条回复')]"
            expand_attempts = 0
            while expand_attempts < 5:
                expand_attempts += 1; clicked_an_expand_button_this_pass = False
                try:
                    visible_expand_buttons = [b for b in driver.find_elements(By.XPATH, expand_reply_xpath) if b.is_displayed()]
                    if not visible_expand_buttons: break
                    print(f"  Found {len(visible_expand_buttons)} visible 'Expand Replies' links.")
                    for button in visible_expand_buttons:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", button); time.sleep(0.4)
                            button_to_click = WebDriverWait(driver, 3).until(EC.element_to_be_clickable(button))
                            driver.execute_script("arguments[0].click();", button_to_click)
                            print(f"    Clicked 'Expand Replies': {button.text[:20]}"); action_taken_this_loop = True; clicked_an_expand_button_this_pass = True; time.sleep(1.5)
                        except (StaleElementReferenceException, TimeoutException, ElementNotInteractableException): continue
                        except Exception as e_expand: print(f"    Error clicking one 'Expand Replies': {e_expand}")
                    if not clicked_an_expand_button_this_pass: break
                except Exception as e_find_expand: print(f"  Error finding 'Expand Replies': {e_find_expand}"); break

            # 2. Scroll down
            print("  Scrolling down...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)

            # 3. Attempt to scrape comments with authors
            # Each comment item is likely a div, e.g., class="comment_item" (needs inspection)
            # Relative to this item, find author and text.
            # Based on previous screenshot: comments are within <div class="comment__list"> which contains <div class="comment_item">
            # And each comment_item has <div class="comment__item__main"> for the actual content.
            comment_item_xpath = "//div[@class='comment__list']/div[contains(@class,'comment_item')]" # XPath for each comment's main container
            try:
                comment_items = driver.find_elements(By.XPATH, comment_item_xpath)
                new_comments_this_pass = 0
                if comment_items:
                    print(f"  Found {len(comment_items)} potential comment item containers.")
                for item_div in comment_items:
                    try:
                        # XPath relative to item_div to find author name/link and comment text
                        # LIKELY NEEDS INSPECTION AND ADJUSTMENT
                        author_name_element = item_div.find_element(By.XPATH, ".//div[@class='comment__item__main']//a[contains(@class,'name')]")
                        comment_author_name = author_name_element.text.strip()
                        author_href = author_name_element.get_attribute('href')
                        comment_author_id = None
                        if author_href:
                            parts = author_href.split('/')
                            comment_author_id = parts[-1] if parts[-1] else parts[-2]

                        comment_text_element = item_div.find_element(By.XPATH, ".//div[@class='comment__item__main']/p")
                        comment_text = comment_text_element.text.strip()

                        # Use a tuple of (author_id, first ~50 chars of text) for uniqueness tracking
                        # This prevents adding the same comment if it's re-rendered or slightly changes non-text attributes
                        comment_signature = (comment_author_id, comment_text[:50])

                        if comment_text and comment_signature not in unique_comments_tracker:
                            unique_comments_tracker.add(comment_signature)
                            scraped_data["comments"].append({
                                "author_name": comment_author_name,
                                "author_id": comment_author_id,
                                "text": comment_text
                            })
                            new_comments_this_pass += 1
                    except NoSuchElementException:
                        # This means the structure within a comment_item was not as expected.
                        # print("    Could not find author/text structure in a comment item.")
                        continue
                    except StaleElementReferenceException:
                        # print("    Stale element within comment item, skipping.")
                        continue
                if new_comments_this_pass > 0:
                    print(f"    Added {new_comments_this_pass} new unique comments with author info.")
                    action_taken_this_loop = True
            except Exception as e_find_comments: print(f"  Error finding comment items: {e_find_comments}")

            # 4. Click "展开查看更多" (Load More Main Comments)
            load_more_comments_xpath = "//a[@class='show_more' and .//span[text()='展开查看更多']]"
            try:
                load_more_button = interaction_wait.until(EC.element_to_be_clickable((By.XPATH, load_more_comments_xpath)))
                print("  Found '展开查看更多' button. Clicking...")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", load_more_button); time.sleep(0.4)
                driver.execute_script("arguments[0].click();", load_more_button)
                print("    Clicked '展开查看更多'."); action_taken_this_loop = True; time.sleep(scroll_pause_time + 0.5)
            except TimeoutException: print("  '展开查看更多' button not found or not clickable (might be all loaded).")
            except Exception as e_load_more: print(f"  Error clicking '展开查看更多': {e_load_more}")

            # 5. Check for loop termination
            current_total_comments_in_set = len(unique_comments_tracker)
            print(f"  Loop {i+1} end. Total unique comments in set: {current_total_comments_in_set}. Previously: {last_total_comments_in_set}")
            if not action_taken_this_loop and current_total_comments_in_set == last_total_comments_in_set:
                print("No actions taken and comment count unchanged. Assuming completion."); break
            last_total_comments_in_set = current_total_comments_in_set
            if i == max_main_loops - 1: print("Reached max main loops.")
            driver.save_screenshot(os.path.join(screenshot_dir,f"main_loop_end_{i+1}.png"))

        print(f"\n--- Finished comment scraping. Total unique comments: {len(scraped_data['comments'])} ---")

    except Exception as e:
        print(f"\n--- An critical error occurred ---"); print(f"Error Type: {type(e).__name__}"); print(f"Error Details: {e}")
        if driver:
            try: error_ss_path = os.path.join(screenshot_dir, "critical_error.png"); driver.save_screenshot(error_ss_path); print(f"Saved critical error screenshot.")
            except Exception as e_ss_crit: print(f"Could not save critical error screenshot: {e_ss_crit}")
    finally:
        if driver: print("Closing the browser..."); driver.quit(); print("Browser closed.")
    return scraped_data

# --- Main execution block ---
if __name__ == "__main__":
    target_url = "https://xueqiu.com/5669998349/334081638"
    print(f"--- Starting Scraper for URL: {target_url} ---")

    data = scrape_post_comments_with_authors(target_url, max_main_loops=20, scroll_pause_time=2.5)

    print("\n" + "="*30); print("      Scraped Data Summary"); print("="*30)
    if data["post_content"]:
        print("\n--- Main Post ---")
        print(f"Author: {data.get('post_author_name')} (ID: {data.get('post_author_id')})")
        print(f"Content: {data['post_content']}")
    else: print("\n>>> Main post content not scraped. <<<")

    if data["comments"]:
        print(f"\n--- Comments ({len(data['comments'])}) ---")
        for i, comment_data in enumerate(data["comments"]):
            print(f"{i+1}. Author: {comment_data['author_name']} (ID: {comment_data['author_id']})")
            print(f"   Comment: {comment_data['text']}")
    else: print("\n>>> No comments were scraped. <<<")
    print("\n" + "="*30); print(f"Check console logs and folder for details.")

--- Starting Scraper for URL: https://xueqiu.com/5669998349/334081638 ---
Setting up WebDriver...
Created 'screenshots_post_authors_comments' directory.
Navigating to: https://xueqiu.com/5669998349/334081638
Article page indicators loaded.
Looking for '跳过' pop-up...
'跳过' pop-up not found/timed out.
Looking for 'X' pop-up...
Finished checking for 'X' pop-ups.
Scraping main post author and content...
Post Author: 麻瓜价投 (ID: 5669998349)
Post content scraped (length: 627).

--- Starting scroll and comment extraction ---
--- Main Loop Iteration #1 ---
  Scrolling down...
  Found '展开查看更多' button. Clicking...
    Clicked '展开查看更多'.
  Loop 1 end. Total unique comments in set: 0. Previously: -1
--- Main Loop Iteration #2 ---
  Scrolling down...
  '展开查看更多' button not found or not clickable (might be all loaded).
  Loop 2 end. Total unique comments in set: 0. Previously: 0
No actions taken and comment count unchanged. Assuming completion.

--- Finished comment scraping. Total unique comments: 0 ---